<a href="https://colab.research.google.com/github/Pipsicle/sit742/blob/master/Copy_of_github_742.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIT742: Modern Data Science 
**(Assessment Task 02: Bank Marketing Data Analytics)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.

Prepared by **SIT742 Teaching Team**


---

**Project Group Information:**

- Names:
- Student IDs:
- Emails:

---

## 1. Import Spark

In [0]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar --extract --file spark-2.4.0-bin-hadoop2.7.tgz
#!pip install pyspark
!pip install findspark
#https://www.apache.org/dyn/closer.lua/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:

!pip install wget 
import wget
import findspark
findspark.init()
from pyspark.sql import SparkSession 
spark = SparkSession.builder.master("local[*]").getOrCreate()
link_to_data = 'https://github.com/tulip-lab/sit742/raw/master/Assessment/2019/data/bank.csv'
DataSet = wget.download(link_to_data)
df = spark.read.csv(DataSet,header=True,sep=",")
#df.show(2)
#print(df.collect())
#df.columns
#df.count()
#df.describe()
#df.fillna(-1).show(5)
df.na.drop()
df.registerTempTable('bank_table')
spark.sql('select age, marital, job, education, balance, default,housing,loan,duration,campaign, previous,poutcome, deposit from bank_table where poutcome IN ("success", "failure") AND education != "unknown" AND job != "unknown"' ).show(2181)

+---+--------+-------------+---------+-------+-------+-------+----+--------+--------+--------+--------+-------+
|age| marital|          job|education|balance|default|housing|loan|duration|campaign|previous|poutcome|deposit|
+---+--------+-------------+---------+-------+-------+-------+----+--------+--------+--------+--------+-------+
| 33| married|     services|secondary|   3444|     no|    yes|  no|     144|       1|       4| failure|    yes|
| 56| married|   technician|secondary|    589|     no|    yes|  no|     518|       1|       2| success|    yes|
| 34| married|       admin.| tertiary|    899|     no|    yes|  no|     114|       1|       3| failure|    yes|
| 53| married|      retired| tertiary|   2269|     no|     no|  no|    1091|       2|       1| success|    yes|
| 37| married|   technician|secondary|   5115|     no|    yes|  no|    1210|       2|       4| failure|    yes|
| 45| married| entrepreneur|secondary|    781|     no|     no| yes|     652|       2|       2| failure| 

## 2. Read and check data

In [0]:
!pip install wget  

In [0]:
import wget

#read data from tulip labs on github and store it as DataSet
link_to_data = 'https://github.com/tulip-lab/sit742/raw/master/Assessment/2019/data/bank.csv'
DataSet = wget.download(link_to_data)

In [0]:
!ls

In [0]:
#reading the csv file into a spark dataframe
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.csv('bank.csv', header = True, inferSchema = True) 

#check import of csv file, show columns and 5 rows
df.printSchema() 
df.show(5)

+---+--------+-------------+-------------+
| id|category|categoryIndex|  categoryVec|
+---+--------+-------------+-------------+
|  0|       a|          0.0|(2,[0],[1.0])|
|  1|       b|          2.0|    (2,[],[])|
|  2|       c|          1.0|(2,[1],[1.0])|
|  3|       a|          0.0|(2,[0],[1.0])|
|  4|       a|          0.0|(2,[0],[1.0])|
|  5|       c|          1.0|(2,[1],[1.0])|
+---+--------+-------------+-------------+



In [0]:
# Check data distribution
df.describe().show()

## 3. Select features

In [0]:
# select features ('age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit') as df2
df2 = df.select('age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit')

#df2.describe().show()

In [0]:
#Remove invalid rows/records using spark.sql

#Creating a local temporary view of df2 for spark.sql as 'bank_table'
df2.createOrReplaceTempView('bank_table')

#Removing rows where poutcome does not equal either 'success' or 'failure'
#Removing any rows where any columns value is equal to 'unknown' (excpet for column poutcome as these values would be removed via 'poutcome IN ("success", "failure")')
df2 = spark.sql('select * from bank_table where poutcome IN ("success", "failure") AND  "unknown" NOT IN (age, job, marital, education, default, balance, housing, loan, campaign, pdays, previous, deposit)')

In [0]:
#Using StringIndexer encode a string column to a column of indices
#Use One Hot Encoder Estimator to map catergoical features to a binary vector
StringColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'poutcome']
StringColumnsIndVec = []

for StringColumn in StringColumns:
    stringIndexer = StringIndexer(inputCol = StringColumn, outputCol = StringColumn + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[StringColumn + "classVec"])
    StringColumnsIndVec += [stringIndexer, encoder]

#print(StringColumnsIndVec)

#use StringIndexer to encode a string column to a column of indices for output variable deposit
#Add deposit to StringColumnsIndVec[]
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
StringColumnsIndVec += [label_stringIdx]

#add numerical columns to StringColumnsIndVec[] 
numericColumns = ['age', 'balance', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in StringColumns] + numericColumns
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
StringColumnsIndVec += [assembler]

#generate df2 column list for selection after pipeline
selectedCols = ['label', 'features'] + cols

#pipeline data to fit and transform assembled columns 
pipeline = Pipeline(stages = StringColumnsIndVec)
pipelineModel = pipeline.fit(df2)
df2 = pipelineModel.transform(df2)

df2 = df2.select(selectedCols)

df2.show()

### 3.1 normalisation

In [0]:
# then apply Min-Max normalisation on each attribute using MinMaxScaler  
from pyspark.ml.feature import MinMaxScaler
  

## 4. Unsupervised learning

### 4.1 K-means

In [0]:
# Perform unsupervised learning on df2 with k-means 
# you can use whole df2 as both training and testing data, 
# evaluate the clustering result using Accuracy.  

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
 

### 4.2 PCA

In [0]:
# generate a scatter plot using the first two PCA components to investigate the data distribution.

from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
 

## 5. Supervised learning

In [0]:
train, test = df2.randomSplit([0.7, 0.3], seed = 742)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

### 5.1 LogisticRegression

In [0]:
# Logistic Regression

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator 

In [0]:
# exam the coefficients

### 5.2 Decision tree

In [0]:
# Decision tree

from pyspark.ml.classification import DecisionTreeClassifier 

### 5.3 NaiveBayes

In [0]:
# NaiveBayes

from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
 